In [1]:
pip install tensorflow scikit-learn


In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Step 3a: Load and preprocess data
def load_data(data_dir):
    data = []
    labels = []
    for plant_folder in os.listdir(data_dir):
        plant_path = os.path.join(data_dir, plant_folder)
        if os.path.isdir(plant_path):
            for img_file in os.listdir(plant_path):
                img_path = os.path.join(plant_path, img_file)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (64, 64))  # Resize images for uniformity
                data.append(img)
                labels.append(plant_folder)
    return np.array(data), np.array(labels)

data_dir = "/content/drive/MyDrive/aitest/plant"
data, labels = load_data(data_dir)

# Step 3b: Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Step 3c: Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data, encoded_labels, test_size=0.2, random_state=42)

# Step 3d: Build the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 3e: Train the model
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow(X_train, y_train, batch_size=32)
test_generator = datagen.flow(X_test, y_test, batch_size=32)

model.fit(train_generator, epochs=10, validation_data=test_generator)


Epoch 1/10
3/3 [==============================] - 2s 279ms/step - loss: 1.4436 - accuracy: 0.2812 - val_loss: 1.1138 - val_accuracy: 0.3333
Epoch 2/10
3/3 [==============================] - 1s 183ms/step - loss: 1.1487 - accuracy: 0.3542 - val_loss: 1.0624 - val_accuracy: 0.4167
Epoch 3/10
3/3 [==============================] - 1s 173ms/step - loss: 1.0847 - accuracy: 0.3646 - val_loss: 1.0997 - val_accuracy: 0.2500
Epoch 4/10
3/3 [==============================] - 1s 173ms/step - loss: 1.0529 - accuracy: 0.5000 - val_loss: 1.0896 - val_accuracy: 0.5000
Epoch 5/10
3/3 [==============================] - 1s 185ms/step - loss: 1.0122 - accuracy: 0.5208 - val_loss: 1.0132 - val_accuracy: 0.4583
Epoch 6/10
3/3 [==============================] - 1s 177ms/step - loss: 0.9204 - accuracy: 0.5938 - val_loss: 0.9842 - val_accuracy: 0.5833
Epoch 7/10
3/3 [==============================] - 1s 169ms/step - loss: 0.8310 - accuracy: 0.6458 - val_loss: 0.8791 - val_accuracy: 0.5000
Epoch 8/10
3/3 [====

In [4]:
def predict_plant(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (64, 64))
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)
    plant_name = label_encoder.classes_[predicted_class]

    return plant_name

# Example usage:
image_path = "/content/drive/MyDrive/aitest/testimg/p001.jpeg"
predicted_plant = predict_plant(image_path)
print(f"The predicted plant is: {predicted_plant}")


1/1 [==============================] - 0s 106ms/step
The predicted plant is: Brown spot
